## **1. Import Modul**

In [1]:
import pandas as pd  
import time

from google_play_scraper import app, Sort, reviews

## **2. Data**

### Scraping data ulasan aplikasi Google Lens

In [2]:
def scrape_reviews(count):
    app_id = 'com.google.ar.lens'
    country = 'id'  
    language = 'id'  
    
    print(f"Memulai scraping ulasan")
    
    all_reviews = []
    continuation_token = None
    batch_size = 1000  # Jumlah ulasan per request
    
    # Fungsi untuk memproses ulasan
    def process_reviews(reviews):
        processed = []
        for r in reviews:
            processed.append({
                'review_id': r['reviewId'],
                'username': r['userName'],
                'rating': r['score'],
                'date': r['at'].strftime('%Y-%m-%d %H:%M:%S'),
                'content': r['content']
            })
        return processed
    
    while len(all_reviews) < count:
        try:
            # Mengambil ulasan
            result, continuation_token = reviews(
                app_id,
                lang=language,
                country=country,
                sort=Sort.MOST_RELEVANT,
                count=batch_size,
                continuation_token=continuation_token
            )
            
            # Memproses ulasan
            batch_processed = process_reviews(result)
            all_reviews.extend(batch_processed)
            
            print(f"Berhasil mengambil {len(batch_processed)} ulasan. Total: {len(all_reviews)}")
            
            # Jika tidak ada token lanjutan, hentikan loop
            if continuation_token is None:
                print("Tidak ada ulasan lagi yang tersedia.")
                break
                
            # Jeda untuk menghindari rate limiting
            time.sleep(3)
            
        except Exception as e:
            print(f"Error: {str(e)}")
            print("Menunggu 10 detik sebelum mencoba lagi.")
            time.sleep(10)
    
    # Membuat DataFrame
    df = pd.DataFrame(all_reviews)
    
    print(f"\nScraping selesai! Total ulasan yang didapat: {len(df)}")
    
    return df

In [ ]:
reviews_df = scrape_reviews(count=13000)

Memulai scraping ulasan
Berhasil mengambil 1000 ulasan. Total: 1000
Berhasil mengambil 1000 ulasan. Total: 2000
Berhasil mengambil 1000 ulasan. Total: 3000
Berhasil mengambil 1000 ulasan. Total: 4000
Berhasil mengambil 1000 ulasan. Total: 5000
Berhasil mengambil 1000 ulasan. Total: 6000
Berhasil mengambil 1000 ulasan. Total: 7000
Berhasil mengambil 1000 ulasan. Total: 8000
Berhasil mengambil 1000 ulasan. Total: 9000
Berhasil mengambil 1000 ulasan. Total: 10000
Berhasil mengambil 1000 ulasan. Total: 11000
Berhasil mengambil 1000 ulasan. Total: 12000
Berhasil mengambil 1000 ulasan. Total: 13000

Scraping selesai! Total ulasan yang didapat: 13000


In [4]:
reviews_df.head(5)

,review_id,username,rating,date,content
0,3a17e409-3930-45e5-807b-92c00c94885b,Rdani Hidayat,1,2024-12-12 21:00:03,"Makin di update makin jelek, banyak fitur bagu..."
1,f20e5384-0688-4a72-b243-2d0eb709b7b2,Trafalgar D Water Law,5,2020-12-29 17:34:13,Banyak membantu langsung mengenali apa yang ma...
2,6e0f2d38-ceb5-4ba1-a6b4-11f60468b656,HERMAWAN. PS,5,2021-11-20 07:53:02,Bisa menterjemahkan foto (langsung dari kamera...
3,51d8da31-e2f8-45a8-852f-e39fd89da951,Irlangga Marlan,2,2023-12-25 23:41:44,Update sekrang sangat tidak memuaskan. Banyak ...
4,a8d87b61-6c64-48c0-b1a3-2f40d98985d8,Agus Surya,4,2025-03-12 16:27:20,Selalu mengandalkan aplikasi google lens untuk...


In [5]:
reviews_df.to_csv("Dataset scraping.csv", index=False)